In [17]:
!pip install -i https://test.pypi.org/simple/ krovetz

Looking in indexes: https://test.pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 2.9 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for krovetz: filename=krovetz-1.0.0-cp310-cp310-linux_x86_64.whl size=289875 sha256=4ad5d47dec79b7771fafc286b1ab84010e3e8be7f987917bbbbe4c892cea31e8
  Stored in directory: /root/.cache/pip/wheels/4b/8f/7b/ba27616be01a2d9b8281a280d25acc77f20ab605368f2d607d
Successfully built krovetz


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import re
import ast
from nltk.stem import PorterStemmer
import warnings
import krovetz
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
contraction_mapping = {"ain't": "is not", "its": "it is","aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [4]:
stop_words = set(stopwords.words('english')) 
def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [18]:
keywords_df=pd.read_csv("/kaggle/input/keywords-title/keywords_title.csv")
keywords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13228 entries, 0 to 13227
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  13228 non-null  int64 
 1   Title       13228 non-null  object
 2   Keywords    13228 non-null  object
dtypes: int64(1), object(2)
memory usage: 310.2+ KB


In [19]:
keywords_df.drop(['Unnamed: 0'],axis=1,inplace=True)
keywords_df['Keywords'] = keywords_df['Keywords'].apply(ast.literal_eval)

In [20]:
for i in range(13228):
    words=keywords_df['Keywords'][i]
    for j in range(len(words)):
        words[j]=list(words[j])
    keywords_df['Keywords'][i]=words
print(keywords_df['Keywords'][10])

[['naruto', 0.6194], ['konohagakure', 0.4758], ['kyuubi', 0.4646], ['ninja', 0.4147], ['hokage', 0.3895], ['demon', 0.3415], ['uzumaki', 0.3361], ['rampage', 0.3155], ['deadly', 0.2892], ['village', 0.2886], ['havoc', 0.2874], ['sacrificed', 0.2837], ['attacked', 0.2537], ['foes', 0.251], ['fox', 0.2509], ['fourth', 0.2478], ['oments', 0.2434], ['monstrous', 0.2417], ['life', 0.2346], ['leader', 0.2302], ['birth', 0.2268], ['nine', 0.2209], ['order', 0.211], ['hidden', 0.2103], ['living', 0.2074]]


In [23]:
ks = krovetz.PyKrovetzStemmer()
for i in range(13228):
    words=keywords_df['Keywords'][i]
    for j in range(len(words)):
        words[j][0]=ks.stem(words[j][0])
    keywords_df['Keywords'][i]=words
print(keywords_df['Keywords'][10])

[['naruto', 0.6194], ['konohagakure', 0.4758], ['kyuubi', 0.4646], ['ninja', 0.4147], ['hokage', 0.3895], ['demon', 0.3415], ['uzumaki', 0.3361], ['rampage', 0.3155], ['deadly', 0.2892], ['village', 0.2886], ['havoc', 0.2874], ['sacrifice', 0.2837], ['attack', 0.2537], ['foe', 0.251], ['fox', 0.2509], ['fourth', 0.2478], ['oment', 0.2434], ['monstrous', 0.2417], ['life', 0.2346], ['leader', 0.2302], ['birth', 0.2268], ['nine', 0.2209], ['order', 0.211], ['hidden', 0.2103], ['living', 0.2074]]


In [26]:
sample_text="ninja saves the village"
sample_text=text_cleaner(sample_text)
results={}
for i in sample_text.split():
    for j in range(13228):
        words=keywords_df['Keywords'][j]
        for k in range(len(words)):
            if(words[k][0]==i):
                if words[k][0] in results.keys() or words[k][0] in results.values():
                    results[keywords_df['Title'][j]]+=words[k][1]
                else:
                    results[keywords_df['Title'][j]]=words[k][1]

In [31]:
sorted_results = dict(sorted(results.items(), key=lambda item: item[1], reverse=True))
sorted_results=list(sorted_results.items())
for i in range(10):
    print(sorted_results[i])

('Ninjutsu Hinotama Kozou: Edo no Maki', 0.6682)
('Ninja Hattori-kun (2012) Special', 0.6369)
('Nintama Rantarou', 0.6275)
('Ninja Slayer From Animation', 0.6231)
('Ninja Bugei-chou', 0.5941)
('Shuranosuke Zanmaken: Shikamamon no Otoko', 0.5908)
('Jigen Sengokushi: Kuro no Shishi - Jinnai-hen', 0.5892)
('Rekka no Honoo', 0.5874)
('Ninja Collection', 0.5772)
('Soccer', 0.5768)
